Initial preparation by Valery Ilin

Modified by: Elvira Zainulina

Post modif: Valery Ilin

In [6]:
import pyrealsense2 as rs
import open3d as o3d
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.transform import Rotation as R
from mpl_toolkits.mplot3d import Axes3D
import time

# Collecting frames for next dev

In [29]:
count_of_frames = 20
frames_between_frames_D435 = 15

In [30]:
count_of_frames * frames_between_frames_D435 / 30

10.0

# Iitial configs

In [31]:
# data filenames
d435_filename = '../data/D435.bag'
t265_filename = '../data/T265.bag'

In [32]:
# config fo D435
cfg_d435 = rs.config()
cfg_d435.enable_device_from_file(d435_filename)
cfg_d435.enable_stream(rs.stream.depth, 848, 480, rs.format.z16, 30)
pipe_d435 = rs.pipeline()

In [33]:
# config fo T265
cfg_t265 = rs.config()
cfg_t265.enable_device_from_file(t265_filename)
cfg_t265.enable_stream(rs.stream.pose)
pipe_t265 = rs.pipeline()


# Get all data from 265

In [34]:
cfg = rs.config()
cfg.enable_device_from_file(t265_filename)
cfg.enable_stream(rs.stream.pose)
pipe = rs.pipeline()
profile = pipe.start(cfg)

t265_data_list = []
t265_time_list = []
first_timestamp = None
while(True):

    frames = pipe.wait_for_frames()
    pose = frames.get_pose_frame()
    
    if pose:
        if pose.get_timestamp() == first_timestamp:
            print('reached the first frame - reading bag file completed')
            break
        if first_timestamp is None:
            first_timestamp = pose.get_timestamp()
            print('initial_timestamp',first_timestamp)
#         print('current-first',pose.get_timestamp()-first_timestamp)
        t265_data_list.append(pose.get_pose_data())
        t265_time_list.append(pose.get_timestamp())
    
pipe.stop()

initial_timestamp 1585059273040.0645
reached the first frame - reading bag file completed


In [35]:
len(t265_data_list)

6146

In [36]:
def get_translation_pose_from_data(pose):
    # https://github.com/IntelRealSense/librealsense/issues/5178
    try:
        trans = pose.translation
#         return np.array([-trans.z, trans.x, -trans.y]) # world(x,y,z) = real(-z,x,-y)
        return np.array([trans.x, trans.y, trans.z]) # world(x,y,z) = real(-z,x,-y)
    except:
        return

In [37]:
t265_frames_all = t265_data_list

In [38]:
get_translation_pose_from_data(t265_frames_all[60])


array([-0.00132552,  0.00054592,  0.00075021])

## Collecting frames

In [39]:
d435_frames = []
pipe_d435.start(cfg_d435)
while len(d435_frames) != count_of_frames:
    for i in range(frames_between_frames_D435):
        depth_frame = pipe_d435.wait_for_frames().get_depth_frame()
    d435_frames.append(depth_frame)

In [40]:
# pipe_t265.start(cfg_t265)

pose_frame = None
pose_frame_time = -1
thresh = 5
t265_frames = []
t265_frames_temp = []
n = 0

for i, depth_frame in enumerate(d435_frames):
    depth_frame_time = depth_frame.get_timestamp()
    while abs(depth_frame_time-pose_frame_time) > thresh:
        pose_frame = pipe_t265.wait_for_frames().get_pose_frame()
        pose_frame_time = pose_frame.get_timestamp()
    while abs(depth_frame_time-pose_frame_time) <= thresh:
        pose_frame = pipe_t265.wait_for_frames().get_pose_frame()
        t265_frames_temp.append(pose_frame)
        pose_frame_time = pose_frame.get_timestamp()

    min_int_index = 0
    min_interval = thresh * 2
    for ind_pose, temp_pose in enumerate(t265_frames_temp):
        if abs(depth_frame_time-temp_pose.get_timestamp())<min_interval:
            min_interval = abs(depth_frame_time-temp_pose.get_timestamp())
            min_int_index = ind_pose
    t265_frames.append(t265_frames_temp[min_int_index])
    print(i)

RuntimeError: wait_for_frames cannot be called before start()

In [19]:
t265_frames

## Test for frames

In [20]:
def count_diff_frames(frames):
    prev_frame = None
    for i, frame in enumerate(frames):
        if prev_frame is not None:
            print("Difference bw {0} and {1} frames is {2:.4f} ms".format(i, i-1, frame.get_timestamp()-prev_frame.get_timestamp()))
        prev_frame = frame

In [21]:
def get_diff_sensors(frames1, frames2):
    if len(frames1) != len(frames2):
        return
    else:
        for i in range(len(frames1)):
            print("Difference bw frames1[{0}] and frames2[{0}] frames is {1:.4f} ms".format(i, frames1[i].get_timestamp()-frames2[i].get_timestamp()))
        

In [22]:
count_diff_frames(d435_frames)
count_diff_frames(t265_frames)

Difference bw 1 and 0 frames is 466.9939 ms
Difference bw 2 and 1 frames is 466.9417 ms
Difference bw 3 and 2 frames is 467.0337 ms
Difference bw 4 and 3 frames is 466.8530 ms
Difference bw 5 and 4 frames is 467.0156 ms
Difference bw 6 and 5 frames is 466.9106 ms
Difference bw 7 and 6 frames is 467.0164 ms
Difference bw 8 and 7 frames is 466.8916 ms
Difference bw 9 and 8 frames is 467.0095 ms
Difference bw 10 and 9 frames is 466.9180 ms
Difference bw 11 and 10 frames is 466.9651 ms
Difference bw 12 and 11 frames is 467.0002 ms
Difference bw 13 and 12 frames is 467.5154 ms
Difference bw 14 and 13 frames is 466.3035 ms
Difference bw 15 and 14 frames is 466.9458 ms
Difference bw 16 and 15 frames is 466.9670 ms
Difference bw 17 and 16 frames is 467.0144 ms
Difference bw 18 and 17 frames is 467.0010 ms
Difference bw 19 and 18 frames is 466.8860 ms
Difference bw 20 and 19 frames is 466.8533 ms
Difference bw 21 and 20 frames is 467.0469 ms
Difference bw 22 and 21 frames is 466.9763 ms
Differe

In [23]:
get_diff_sensors(d435_frames, t265_frames)

Difference bw frames1[0] and frames2[0] frames is -4.2820 ms
Difference bw frames1[1] and frames2[1] frames is -2.2205 ms
Difference bw frames1[2] and frames2[2] frames is -5.1963 ms
Difference bw frames1[3] and frames2[3] frames is -3.1255 ms
Difference bw frames1[4] and frames2[4] frames is 1863.5403 ms
Difference bw frames1[5] and frames2[5] frames is -4.2468 ms
Difference bw frames1[6] and frames2[6] frames is -2.1404 ms
Difference bw frames1[7] and frames2[7] frames is -5.0188 ms
Difference bw frames1[8] and frames2[8] frames is -3.1948 ms
Difference bw frames1[9] and frames2[9] frames is -5.9197 ms
Difference bw frames1[10] and frames2[10] frames is -4.1384 ms
Difference bw frames1[11] and frames2[11] frames is 5132.2671 ms
Difference bw frames1[12] and frames2[12] frames is -4.8882 ms
Difference bw frames1[13] and frames2[13] frames is -2.2705 ms
Difference bw frames1[14] and frames2[14] frames is -5.9277 ms
Difference bw frames1[15] and frames2[15] frames is -3.9214 ms
Differen

# Analysis

## Helpers

In [24]:
import time
from functools import wraps

def timing(f):
    @wraps(f)
    def wrap(*args, **kw):
        time_start = time.time()
        result = f(*args, **kw)
        time_end = time.time()
        print('----------func:%r took: %2.4f sec' % (f.__name__, time_end-time_start))
        return result
    return wrap

In [ ]:
tm_T265toD435 = np.array([[0.999968402, -0.006753626, -0.004188075, -0.015890727],
                          [-0.006685408, -0.999848172, 0.016093893, 0.028273059],
                          [-0.004296131, -0.016065384, -0.999861654, -0.009375589],
                          [0, 0, 0, 1]])

In [ ]:
# Graph plot
t265 = np.load('../logs/points_trajectory_T265.npy') #np array coords
d435 = np.load('../logs/points_trajectory_D435.npy')/1 # np array coords
# before = 10
before = d435.shape[0]
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.plot(t265[:before, 0], t265[:before, 1], t265[:before, 2], label='t265')
ax.plot(d435[:before, 0], d435[:before, 1], d435[:before, 2], label='d435')
ax.view_init(elev=22, azim=80)
plt.legend()

## Functions

In [ ]:
def get_transformation265(pose):
    data = pose.get_pose_data()
    data_rot = [float(i.strip('xyzw: ')) for i in str(data.rotation).split(', ')]
    r = R.from_quat(data_rot)
    rotation = np.array(r.as_dcm())
    translation = np.array([float(i.strip('xyzw: ')) for i in str(data.translation).split(', ')])[np.newaxis].T
    T = np.hstack((rotation, translation))
    T = np.vstack((T, np.array([0, 0, 0, 1])))
    return T

In [ ]:
def get_translation_pose(pose):
    # https://github.com/IntelRealSense/librealsense/issues/5178
    try:
        data = pose.get_pose_data()
        trans = data.translation
        return np.array([trans.x, trans.y, trans.z]) # world(x,y,z) = real(-z,x,-y)
#         return np.array([-trans.z, trans.x, -trans.y]) # world(x,y,z) = real(-z,x,-y)
    except:
        return

In [ ]:
get_translation_pose(t265_frames[0])

In [ ]:
t265_translations = np.array([get_translation_pose(pose) for pose in t265_frames])
t265_translations

In [ ]:
get_transformation265(t265_frames[0])

In [ ]:
def apply_transformation(transformation, points):
    """

    :param transformation: 4x4 np.array
    :param points: Nx3 np.array
    :return: transformed Nx3 np.array
    """
    if transformation is None or points is None:
        return None
    else:
        coordinates = np.hstack((points, np.ones((points.shape[0], 1))))
        return (transformation @ coordinates.T).T[:, :-1]

In [ ]:
def get_coordinates(depth_frame, make_sampling=True, koef = 2**2):
    """
     TODO
    :param make_sampling:
    :return:
    """
    pc = rs.pointcloud()
    if make_sampling:
        decimate = rs.decimation_filter()
        decimate.set_option(rs.option.filter_magnitude, koef)
        depth_frame = decimate.process(depth_frame)

        points = pc.calculate(depth_frame).as_points()
    else:
        points = pc.calculate(depth_frame).as_points()

    coordinates = np.ndarray(buffer=points.get_vertices(), dtype=np.float32, shape=(points.size(), 3))
    coordinates = coordinates[coordinates[:, 2] != 0]

    coordinates = apply_transformation(tm_T265toD435, coordinates)
    return coordinates


In [ ]:
get_coordinates(d435_frames[0]).shape

In [ ]:
def convert_to_pcl(points):
    """
        TODO
    :param points: Nx3
    :return:
    """
    if points is None:
        return None
    else:
        return o3d.geometry.PointCloud(o3d.utility.Vector3dVector(points))

In [ ]:
convert_to_pcl(get_coordinates(d435_frames[0]))

In [ ]:
# TODO this 100%

def get_transformation435(self, max_point_pair_dist=0.2, init_guess=np.eye(4)):
    old_point_cloud = self.apply_transformation(self.prev_tm, self.point_cloud)
    old_pcl = self.convert_to_pcl(old_point_cloud) # n-1
    self.point_cloud = self.apply_transformation(self.prev_tm, self.get_coordinates())
    if old_pcl is None:
        self.prev_tm = init_guess
        print("\n\n\n{}\n\n\n".format(init_guess))
        return None

    new_pcl = self.convert_to_pcl(self.point_cloud) # n

    tr_mx = o3d.registration.registration_icp(old_pcl, new_pcl, max_point_pair_dist,
                                              self.prev_tm, o3d.registration.TransformationEstimationPointToPoint())\
        .transformation
    self.prev_tm = tr_mx
    return tr_mx

In [ ]:
def plot_PC(Y, TX, ax=None, plot_lines=True):
    ax_is_None = False
    if ax is None:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax_is_None = True
    ax.plot(Y[:,0], Y[:,1], Y[:,2], 'o', label='source points')
    ax.plot(TX[:,0], TX[:,1], TX[:,2], 'o', label='dest points')
    if plot_lines:
        for i in range(Y.shape[0]):
            ax.plot([Y[i,0], TX[i,0]], [Y[i,1], TX[i,1]], [Y[i,2], TX[i,2]], 
                    'g--')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    ax.legend()
    
    if ax_is_None:
        plt.tight_layout()
        plt.show()

# Visualisation

In [ ]:
d435_frames

In [ ]:
t265_frames

In [ ]:
%matplotlib inline
from IPython.display import clear_output

for i in range(len(d435_frames)):
    clear_output(wait=True)
    plt.imshow(d435_frames[i].get_data())
    plt.show()

In [ ]:
points = [get_coordinates(depth) for depth in d435_frames]
pcl = [convert_to_pcl(point) for point in points]

In [ ]:
@timing
def registration_icp(prev_pc, new_pc, thresh):
    """
    TODO
    """
    return o3d.registration.registration_icp(prev_pc, new_pc, thresh)

In [ ]:
%matplotlib inline
regs = []
tr_mxs = []

for i in range(1, len(points)):
    reg = registration_icp(pcl[i-1], pcl[i], 5)
#     print(i, reg.fitness, reg.inlier_rmse)
    tr_mx = reg.transformation
    regs.append(reg)
    tr_mxs.append(tr_mx)

In [ ]:
fitness_points = [reg.fitness for reg in regs]
inlier_rmse_points = [reg.inlier_rmse for reg in regs]

In [ ]:
plt.plot(fitness_points)

In [ ]:
plt.plot(inlier_rmse_points)

In [ ]:
def visualise_transf(points0, points1, reg, size=30):
    fig = plt.figure(figsize=(10, 5))
    ax1 = fig.add_subplot(121, projection='3d')
    ax2 = fig.add_subplot(122, projection='3d')
    cor = np.asarray(reg.correspondence_set)
    set0 = points0[cor[:, 0]]
    set1 = points1[cor[:, 1]]
    inds = np.random.choice(len(cor), size=size, replace=False)
    set0 = set0[inds]
    set1 = set1[inds]
    plot_PC(set0, set1, ax1)
    Tset0 = (reg.transformation @ (np.vstack((set0.T, np.ones(set0.shape[0]))))).T
    plot_PC(Tset0, set1, ax2)
    plt.tight_layout()
    plt.show()
    return set0, set1, Tset0


In [ ]:
set0, set1, Tset0 = visualise_transf(points[0], points[1], regs[0])

In [ ]:
trajectory = [np.zeros(3)]
R_ = np.eye(3)
poses = [R_]

In [ ]:
for i in range(len(tr_mxs)):
    t_est = tr_mxs[0][:3, -1]
    R_est = tr_mxs[0][:3, :3]

    R_ = R_est @ R_
    t = trajectory[-1] + (np.linalg.inv(R_) @ t_est).ravel()
    
    trajectory.append(t) #-t moves point cloud into (0,0) of the initial state 
    poses.append(R_) #R_^-1 translates point cloud into the state of initial frame

In [ ]:
trajectory = np.array(trajectory)
plt.plot(trajectory[:, 1], trajectory[:, 2])

In [ ]:
poses

In [ ]:
def plot_pose(c, pose, ax):
    for i, color in enumerate(['r', 'g', 'b']):
        ax.plot([c[0], pose[0, i]+c[0]], 
                [c[1], pose[1, i]+c[1]],
                [c[2], pose[2, i]+c[2]], color)
    
def plot_poses(trajectory, poses):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot(trajectory[:,0], trajectory[:, 1], trajectory[:,2], 'k')
    for i, pose in enumerate(poses):
        plot_pose(trajectory[i], pose, ax)
#     ax.plot(points_trajectory_T265[:,0], points_trajectory_T265[:, 1], points_trajectory_T265[:, 2])
    plt.tight_layout()
    plt.show()

In [ ]:
plot_poses(np.array(trajectory), np.array(poses))

In [ ]:
for i in range(len(tr_mxs)):
    _ = visualise_transf(points[i], points[i+1], regs[i])

In [ ]:
np.mean(points[0], axis=0)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
tr = []
for i in range(len(points)):
#     ax.plot(points[i][:50,0], points[i][:50,1], points[i][:50,2], 'o', label=str(i))
    tr.append(np.mean(points[i], axis=0))
tr = np.array(tr)
ax.plot(tr[:,0], tr[:, 1], tr[:, 2])
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
# ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
points_trajectory_T265 = [[0, 0, 0]]
for pose in t265_frames:
    tr_mx = get_transformation265(pose)
    points_trajectory_T265.append(tr_mx[:3, -1])
points_trajectory_T265 = np.array(points_trajectory_T265)

In [ ]:
plt.scatter(points_trajectory_T265[:, 1], points_trajectory_T265[:, 2])

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
# ax.plot(tr[:,0], tr[:, 1], tr[:, 2])
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
# ax.legend()
ax.plot(points_trajectory_T265[:,0], points_trajectory_T265[:, 1], points_trajectory_T265[:, 2])
plt.tight_layout()
plt.show()

# Visualisation
```
pip install –upgrade pip
pip install PyQT5
```

In [ ]:
tr_mxs_real = [tr_mxs[0]]
for i in range(1, len(tr_mxs)):
    tr_mxs_real.append(tr_mxs_real[-1] @ (tr_mxs[i]))

In [ ]:
tr_points = [point[:3,-1] for point in tr_mxs_real]

In [ ]:
tr_points = np.array(tr_points) * -1

In [ ]:
mark_tr_e = [trajectory.tolist().index(i) for i in trajectory.tolist()]
print(mark_tr_e)
mark_tr = [tr_points.tolist().index(i) for i in tr_points.tolist()]
print(mark_tr)

mark_t265 = [t265_translations.tolist().index(i) for i in t265_translations.tolist()]
print(mark_t265)


In [ ]:
full_t265_trajectory = np.array([get_translation_pose_from_data(pose) for pose in t265_frames_all])

In [ ]:
%matplotlib qt
fig = plt.figure(figsize=(10,10), dpi=100)
ax = fig.add_subplot(111, projection='3d')
# ax.plot(tr[:,0], tr[:, 1], tr[:, 2])
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
# ax.legend()
# ax.plot(trajectory[:,0], trajectory[:, 1], trajectory[:, 2], label='d435_Elvira')
# for i, txt in enumerate(mark_tr_e):
#     ax.text(trajectory[i,0],trajectory[i,1],trajectory[i,2], txt)

ax.plot(tr_points[:,0], tr_points[:, 1], tr_points[:, 2], label='d435_Elvira(post Valera)')
for i, txt in enumerate(mark_tr):
    ax.text(tr_points[i,0],tr_points[i,1],tr_points[i,2], txt)
    
ax.plot(t265_translations[:,0],t265_translations[:,1],t265_translations[:,2], label='t265')
for i, txt in enumerate(mark_t265):
    ax.text(t265_translations[i,0],t265_translations[i,1],t265_translations[i,2], txt)
    
# ax.plot(full_t265_trajectory[:,0],full_t265_trajectory[:,1],full_t265_trajectory[:,2], label='Full trajectory')

plt.tight_layout()
plt.legend()
plt.show()